In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define the local path to your saved model directory
model_path = '../FinalProject/GenshinModel_save/'  # Update this path to your local directory

# Load the model from the local path
model = AutoModelForSequenceClassification.from_pretrained(model_path)

#Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained('../FinalProject/GenshinModel_save/')

In [3]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import tkinter as tk
# from tkinter import messagebox

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def test_review(review_text, model, tokenizer, max_length):
#     #tokenize and preprocess the review
#     encoded_review = tokenizer.encode_plus(
#         review_text,
#         add_special_tokens=True,
#         max_length=max_length,
#         padding='max_length',
#         truncation=True,
#         return_tensors='pt'
#     )
#     #extract token IDs and attention mask
#     input_ids = encoded_review['input_ids']
#     attention_mask = encoded_review['attention_mask']

#     #ensure the tensors are on the appropriate device
#     input_ids = input_ids.to(device)
#     attention_mask = attention_mask.to(device)

#     #make predictions using the model
#     with torch.no_grad():
#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits

#     #get the predicted sentiment (argmax of the logits)
#     predicted_sentiment = torch.argmax(logits, dim=1).item()

#     #map predicted label to sentiment category (0: bad, 1: neutral, 2: good)
#     sentiment_mapping = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE"}
#     predicted_label= sentiment_mapping[predicted_sentiment]

#     return predicted_sentiment, predicted_label

# def submit():
#     review_text = text_entry.get("1.0", "end-1c")  # Get text from Text widget
#     predicted_sentiment, predicted_label = test_review(review_text, model, tokenizer, max_length=64)
#     messagebox.showinfo("Predicted Sentiment", f"Sentiment: {predicted_sentiment}\nLabel: {predicted_label}")

# #Create the GUI
# root = tk.Tk()
# root.title("Review Sentiment Analysis")

# text_entry = tk.Text(root, width=50, height=20)  # Use Text widget with specified height
# text_entry.pack()

# submit_button = tk.Button(root, text="Submit", command=submit)
# submit_button.pack()

# root.mainloop()


In [4]:
# import torch
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import tkinter as tk
# from tkinter import messagebox

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# def test_review(review_text, model, tokenizer, max_length):
#     #tokenize and preprocess the review
#     encoded_review = tokenizer.encode_plus(
#         review_text,
#         add_special_tokens=True,
#         max_length=max_length,
#         padding='max_length',
#         truncation=True,
#         return_tensors='pt'
#     )
#     #extract token IDs and attention mask
#     input_ids = encoded_review['input_ids']
#     attention_mask = encoded_review['attention_mask']

#     #ensure the tensors are on the appropriate device
#     input_ids = input_ids.to(device)
#     attention_mask = attention_mask.to(device)

#     #make predictions using the model
#     with torch.no_grad():
#         outputs = model(input_ids, attention_mask=attention_mask)
#         logits = outputs.logits

#     #get the predicted sentiment (argmax of the logits)
#     predicted_sentiment = torch.argmax(logits, dim=1).item()

#     #map predicted label to sentiment category (0: bad, 1: neutral, 2: good)
#     sentiment_mapping = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE"}
#     predicted_label= sentiment_mapping[predicted_sentiment]

#     return predicted_sentiment, predicted_label

# def submit():
#     review_text = text_entry.get()
#     predicted_sentiment, predicted_label = test_review(review_text, model, tokenizer, max_length=64)
#     messagebox.showinfo("Predicted Sentiment", f"Sentiment: {predicted_sentiment}\nLabel: {predicted_label}")

# #Create the GUI
# root = tk.Tk()
# root.title("Review Sentiment Analysis")

# text_entry = tk.Entry(root, width=50)
# text_entry.pack()

# submit_button = tk.Button(root, text="Submit", command=submit)
# submit_button.pack()

# root.mainloop()


In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import tkinter as tk

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def test_review(review_text, model, tokenizer, max_length):
    #tokenize and preprocess the review
    encoded_review = tokenizer.encode_plus(
        review_text,
        add_special_tokens=True,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    #extract token IDs and attention mask
    input_ids = encoded_review['input_ids']
    attention_mask = encoded_review['attention_mask']

    #ensure the tensors are on the appropriate device
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)

    #make predictions using the model
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    #get the predicted sentiment (argmax of the logits)
    predicted_sentiment = torch.argmax(logits, dim=1).item()

    #map predicted label to sentiment category (0: bad, 1: neutral, 2: good)
    sentiment_mapping = {0: "NEGATIVE", 1: "NEUTRAL", 2: "POSITIVE"}
    predicted_label= sentiment_mapping[predicted_sentiment]

    return predicted_sentiment, predicted_label

def submit():
    review_text = text_entry.get()
    predicted_sentiment, predicted_label = test_review(review_text, model, tokenizer, max_length=64)
    result_label.config(text=f"Sentiment: {predicted_sentiment}\nLabel: {predicted_label}")

#Create the GUI
root = tk.Tk()
root.geometry("400x200")    
root.title("Review Sentiment Analysis")

text_entry = tk.Entry(root, width=50, highlightthickness=2, highlightcolor="blue", highlightbackground="blue")
text_entry.pack(pady=20)

submit_button = tk.Button(root, text="Submit", command=submit)
submit_button.pack()

result_label = tk.Label(root, text="")
result_label.pack(pady=20)

root.mainloop()